In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
data = None
test_data = None
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        if filename == 'train.csv':
            data = pd.read_csv(path)
        if filename == 'test.csv':
            test_data = pd.read_csv(path)
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test_data

In [ ]:
data.dtypes

In [ ]:
data = data.drop(columns='Date')
data = data.drop(columns='StateHoliday')
data

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(data['Sales'], data['Customers'])
# Adding Title to the Plot
plt.title("Sales Based on customers")

In [ ]:
plt.plot(data['Sales']/4)
plt.plot(data['Customers'])
# Adding Title to the Plot
plt.title("Sales Based on customers")

In [ ]:
plt.scatter(data['SchoolHoliday'], data['Customers'])
# Adding Title to the Plot
plt.title("Sales Based Hollidays")

In [ ]:
plt.plot(data['Customers']/1000)
plt.plot(data['SchoolHoliday'])
# Adding Title to the Plot
plt.title("Sales Based on customers")

In [ ]:
test_data = test_data.drop(columns='Date')
test_data = test_data.drop(columns='StateHoliday')
test_data

In [ ]:
plt.scatter(data['Promo'], data['Customers'])
# Adding Title to the Plot
plt.title("Sales Based Promo")

In [ ]:
from sklearn import preprocessing
    
preprocessing.normalize(data)

In [ ]:
import numpy as np
x = data.loc[:, data.columns != 'Sales'].to_numpy()
y = data.loc[:, 'Sales'].to_numpy()
print(x, y.shape)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import random

class Custom(keras.layers.Layer):
    
    def __init__(self, units=32):
        super(Custom, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs, training=None):
        
        out = tf.matmul(inputs, self.w)+self.b
        
        if training:
            pass
        
        return out

In [ ]:
class CustomModel(keras.Sequential):
    
    def __init__(self, layers=None, name=None):
        super().__init__(layers, name)
    
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            
            if random.uniform(0, 1) > 0.5:
                loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
            else:
                loss = self.compiled_loss(x, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}


In [ ]:

if False:
    model = keras.Sequential([
        # the hidden ReLU layers
        layers.Dense(units=10, activation='relu', input_shape=x.shape),
        layers.Dense(units=5, activation='relu'),
        layers.Dense(units=10, activation='relu'),
        # the linear output layer 
        layers.Dense(units=len(y.shape),)])

    model.compile(
    optimizer='adam',
    loss= 'mae',
    metrics = ['accuracy'])

    history = model.fit(
        x,
        y,
        batch_size=len(x),
        epochs=100
    )
elif False:
    model = keras.Sequential([
        # the hidden ReLU layers
        layers.Dense(units=100, activation='relu', input_shape=x.shape),
        layers.Dense(units=50, activation='relu'),
        # the linear output layer 
        layers.Dense(units=len(y.shape),)])

    model.compile(
    optimizer='adam',
    loss= 'mae',
    metrics = ['accuracy'])
    
    history = model.fit(
        x,
        y,
        batch_size=len(x),
        epochs=100
    )
else:
    
    
    l1 = layers.Dense(units=3, activation='relu', input_shape=x.shape)
    l2 = layers.Dense(units=5, activation='sigmoid')
    l3 = layers.Dense(units=len(y.shape))
    
    model = CustomModel([l1, l2, l3])
    
    model.compile(
    optimizer='adam',
    loss= 'mae',
    metrics = ['accuracy'])
    
    history = model.fit(
        x,
        y,
        batch_size=len(x),
        epochs=100
    )
    

In [ ]:
predictions = model.predict(test_data.to_numpy())
prediction_data = pd.DataFrame(predictions)
prediction_data[0]
#predictions = keras.cast(yPred,"float32")

In [ ]:
output = pd.DataFrame({'Id': test_data.Id, 'Sales': prediction_data[0]})
output.to_csv('submission.csv', index=False)
print("Done")